In [ ]:
import os
import re
import time
import datetime
from tqdm import tqdm 
import pandas as pd
import requests

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd()  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

from config import urls_1140412, column_names
from utils import fetch_data, combined_df, parse_admin_region, parse_sale_period, find_first_sale_time, sales_start_time

In [ ]:
import_name = 'urls_1140412'
input_time = import_name.split('_')[1]  # 提取 "1140412" 部分

df = combined_df(urls_1140412, input_time)

In [ ]:
if not os.path.exists('../data'):
    os.makedirs('../data')
    
# 使用今天的日期作為檔名的一部分
today = datetime.datetime.now().strftime('%Y%m%d')
df.to_csv(f'../data/presale_rawdata_{today}.csv', index=False, encoding='utf-8-sig')
df.to_pickle(f'../data/presale_rawdata_{today}.pkl')

In [ ]:
df = pd.read_pickle('../data/presale_rawdata_20250417.pkl')

In [ ]:
df = df.rename(columns=column_names, inplace=False)

In [ ]:
proc_df = df.copy()
# 在 proc_df 裡面新增一個「行政區」欄位
proc_df["行政區"] = proc_df["坐落街道"].apply(parse_admin_region)
proc_df["自售期間"], proc_df["代銷期間"] = zip(*proc_df["銷售期間"].apply(parse_sale_period))

# 依規則從「自售期間」及「代銷期間」欄位提取出7位數字，
# 分別存入新欄位「自售起始時間」與「代銷起始時間」
proc_df["自售起始時間"] = proc_df["自售期間"].apply(find_first_sale_time)
proc_df["代銷起始時間"] = proc_df["代銷期間"].apply(find_first_sale_time)

# 建立「銷售起始時間」欄位
proc_df["銷售起始時間"] = proc_df.apply(sales_start_time, axis=1)

In [ ]:
final_df = proc_df[["縣市", "行政區", "起造人", "社區名稱", "戶數", "銷售起始時間", "銷售期間",  
"編號", "自售起始時間", "代銷起始時間", "備查完成日期", "建照核發日", "坐落街道", "主要用途", "使用分區", "建照執照", "經度", "緯度"]]
final_df

In [ ]:
today = datetime.datetime.now().strftime('%Y%m%d')
final_df.to_pickle(f'../data/presale_output_{today}.pkl')